<a href="https://colab.research.google.com/github/darshan-kale-dsi/Team-05-Bank-Marketing/blob/seb_branch/notebooks/model_adjustment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Model Adjustment

In [76]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder

In [28]:
dataframe = pd.read_csv('bank-full.csv', sep = ';')

Stuff to carry out:
- job, marital, education, contact to be one-hot-encoded
- balance, duration: trim outliers
- pdays to be converted into binary (previously reached or not)
- day and month to be represnted by cosine and sine?
- poutcome?

In [29]:
# trimming extreme values (outliers)
dataframe = dataframe[(dataframe['balance']<10000) & (dataframe['duration']<1800)]

In [30]:
dataframe['pdays'] = dataframe['pdays'].apply(lambda value: 0 if value==-1 else 1 )

<ipython-input-30-cb4f8f792b0f>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe['pdays'] = dataframe['pdays'].apply(lambda value: 0 if value==-1 else 1 )


In [37]:
dataframe['default'] = dataframe['default'].apply(lambda value: 1 if value =='yes' else 0)

In [41]:
dataframe['housing'] = dataframe['housing'].apply(lambda value: 1 if value =='yes' else 0)

In [45]:
dataframe['loan'] = dataframe['loan'].apply(lambda value: 1 if value =='yes' else 0)

In [ ]:
dataframe['y']= dataframe['y'].apply(lambda value: 1 if value =='yes' else 0)

In [68]:
dataframe['month'] = dataframe['month'].replace({'may':5,
                            'jun':6,
                            'jul':7,
                            'aug':8,
                            'oct':10,
                            'nov':11,
                            'dec':12,
                            'jan':1,
                            'feb':2,
                            'mar':3,
                            'apr':4,
                            'sep':9})

<ipython-input-68-9fca51a4bb4e>:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  dataframe['month'] = dataframe['month'].replace({'may':5,


In [69]:
import numpy as np

dataframe['day_sine'] = dataframe['day'].apply(lambda value: np.sin(2*value*np.pi/31))
dataframe['day_cosine'] = dataframe['day'].apply(lambda value: np.cos(2*value*np.pi/31))

dataframe['month_sine'] = dataframe['month'].apply(lambda value: np.sin(2*value*np.pi/12))
dataframe['month_cosine'] = dataframe['month'].apply(lambda value: np.cos(2*value*np.pi/12))

In [71]:
dataframe = dataframe.drop(['day', 'month'], axis = 1)

In [75]:
dataframe

,age,job,marital,education,default,balance,housing,loan,contact,duration,campaign,pdays,previous,poutcome,y,day_sine,day_cosine,month_sine,month_cosine
0,58,management,married,tertiary,0,2143,1,0,unknown,261,1,0,0,unknown,no,0.848644,0.528964,0.5,-0.866025
1,44,technician,single,secondary,0,29,1,0,unknown,151,1,0,0,unknown,no,0.848644,0.528964,0.5,-0.866025
2,33,entrepreneur,married,secondary,0,2,1,1,unknown,76,1,0,0,unknown,no,0.848644,0.528964,0.5,-0.866025
3,47,blue-collar,married,unknown,0,1506,1,0,unknown,92,1,0,0,unknown,no,0.848644,0.528964,0.5,-0.866025
4,33,unknown,single,unknown,0,1,0,0,unknown,198,1,0,0,unknown,no,0.848644,0.528964,0.5,-0.866025
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45206,51,technician,married,tertiary,0,825,0,0,cellular,977,3,0,0,unknown,yes,-0.299363,-0.954139,-0.5,0.866025
45207,71,retired,divorced,primary,0,1729,0,0,cellular,456,2,0,0,unknown,yes,-0.299363,-0.954139,-0.5,0.866025
45208,72,retired,married,secondary,0,5715,0,0,cellular,1127,5,1,3,success,yes,-0.299363,-0.954139,-0.5,0.866025
45209,57,blue-collar,married,secondary,0,668,0,0,telephone,508,4,0,0,unknown,no,-0.299363,-0.954139,-0.5,0.866025


,age,job,marital,education,default,balance,housing,loan,contact,duration,campaign,pdays,previous,poutcome,y,day_sine,day_cosine,month_sine,month_cosine
0,58,management,married,tertiary,0,2143,1,0,unknown,261,1,0,0,unknown,no,0.848644,0.528964,0.5,-0.866025
1,44,technician,single,secondary,0,29,1,0,unknown,151,1,0,0,unknown,no,0.848644,0.528964,0.5,-0.866025
2,33,entrepreneur,married,secondary,0,2,1,1,unknown,76,1,0,0,unknown,no,0.848644,0.528964,0.5,-0.866025
3,47,blue-collar,married,unknown,0,1506,1,0,unknown,92,1,0,0,unknown,no,0.848644,0.528964,0.5,-0.866025
4,33,unknown,single,unknown,0,1,0,0,unknown,198,1,0,0,unknown,no,0.848644,0.528964,0.5,-0.866025
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45206,51,technician,married,tertiary,0,825,0,0,cellular,977,3,0,0,unknown,yes,-0.299363,-0.954139,-0.5,0.866025
45207,71,retired,divorced,primary,0,1729,0,0,cellular,456,2,0,0,unknown,yes,-0.299363,-0.954139,-0.5,0.866025
45208,72,retired,married,secondary,0,5715,0,0,cellular,1127,5,1,3,success,yes,-0.299363,-0.954139,-0.5,0.866025
45209,57,blue-collar,married,secondary,0,668,0,0,telephone,508,4,0,0,unknown,no,-0.299363,-0.954139,-0.5,0.866025


In [77]:
X = dataframe.drop('y', axis = 1)
y = dataframe['y']

X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size = 0.3,
                                                    random_state = 123)

In [85]:
numeric_features = ['age','balance','duration', 'campaign','previous']
categorical_features = ['job', 'marital', 'education', 'contact', 'poutcome']

In [86]:
preprocessor = ColumnTransformer(transformers=[
    ('num', StandardScaler(), numeric_features),
    ('cat', OneHotEncoder(), categorical_features)
])

In [88]:
dataframe_transformed = preprocessor.fit_transform(X_train)

In [90]:
pd.DataFrame(dataframe_transformed)

,0,1,2,3,4,5,6,7,8,9,...,21,22,23,24,25,26,27,28,29,30
0,1.531856,-0.287808,-0.516064,0.073382,-0.304281,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
1,1.057744,2.192679,-0.108780,-0.245733,-0.304281,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
2,-0.933526,-0.212624,-0.906553,7.093924,-0.304281,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
3,-0.554237,1.546092,2.326527,-0.245733,-0.304281,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
4,0.109520,-0.522384,-0.230546,-0.564849,-0.304281,0.0,0.0,0.0,1.0,0.0,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30988,-0.554237,-0.235480,0.084365,-0.245733,-0.304281,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
30989,-1.123171,-0.778613,0.680595,0.392498,-0.304281,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
30990,-0.459414,-0.462838,-0.553854,-0.245733,-0.304281,0.0,0.0,1.0,0.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
30991,-0.743881,-0.307657,-0.385901,-0.245733,0.229153,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0


In [ ]:
ohe_feature_names = preprocessor.named_transformers_['cat'].get_feature_names_out(categorical_features)
final_columns = numeric_features + list(ohe_feature_names)
X_train_preprocessed_df = pd.DataFrame(X_train_preprocessed, columns=final_columns)
X_train_preprocessed_df

In [ ]:
numerical_var = ['coord_x', 'coord_y', 'ffmc', 'dmc', 'dc', 'isi','temp', 'rh', 'wind', 'rain']
categorical_var =['month','day']

preproc1 = ColumnTransformer(transformers = [('num_var',RobustScaler(),numerical_var),
                                             ('cat_var',OneHotEncoder(handle_unknown='infrequent_if_exist'), categorical_var)])

preproc1

In [ ]:
pipeline_a = Pipeline(steps = [('preprocessing',preproc1),
                               ('regressor',baseline_reg)])